In [2]:
pip install -q https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_scibert-0.5.4.tar.gz && pip install wordcloud && pip install fuzzywuzzy nltk --quiet && pip install transformers -U --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy-transformers 1.3.5 requires transformers<4.37.0,>=3.4.0, but you have transformers 4.41.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Load the tokenizer for en_core_sci_scibert
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
# Load extractor class
extractor = Extractor_BERT()
# Load the model SELECT WHICH MODEL
extractor.load_model('en_core_sci_scibert')

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


In [5]:
# Load the list of abtracts, select datasource, extractor, number of abstracts and batch size
abstract_list_cleaned, abstract_ids = load_abstracts_clean_BERT(
    '/root/DS_Evidence/ds_project_folder/data/raw_data/CHEMNER/training.abstracts.txt', 
    extractor,  
    max_rows=1000,
    clean_data=True
)
# Store the model name
model_name = extractor.get_model_name()
# Load annotations
annotation_list_cleaned = load_annotations(
    '/root/DS_Evidence/ds_project_folder/data/raw_data/CHEMNER/training.annotations.txt', 
    abstract_ids, 
    clean_data=True
)

Rows processed: 1000

In [15]:
# Run performance metrics
num_common_elements, precision, recall, f1_score, annotation_unique_values, abstract_unique_values, intersection = calculate_evaluation_metrics(annotation_list_cleaned, abstract_list_cleaned)
# Print performance metrics
print_evaluation_metrics(annotation_list_cleaned, abstract_list_cleaned, abstract_ids)

Number of abstracts scanned: 100
Number of entities extracted: 6474
Number of annotations in dataset: 742
Number of common elements: 0
Precision: 0.0
Recall: 0.0
F1 Score: 0


In [6]:
abstract_file_name='training.abstracts'

In [7]:
# export entities from model
export_entities(model_name, abstract_unique_values, abstract_list_cleaned, abstract_file_name)
# append performance metrics
export_performance(model_name, abstract_list_cleaned, annotation_unique_values, abstract_unique_values, num_common_elements, precision, recall, f1_score,abstract_file_name)

Number of abstracts scanned: 1000
Number of entities extracted: 70489
Number of annotations in dataset: 8502
Number of common elements: 10
Precision: 0.0011761938367442955
Recall: 0.00014186610676843195
F1 Score: 0.0002531934017799496


In [8]:
# Calculate evaluation metrics with fuzzy matches
num_common_elements_fuzzy, precision_fuzzy, recall_fuzzy, f1_score_fuzzy = calculate_evaluation_metrics_with_fuzzy(annotation_list_cleaned, abstract_list_cleaned, filtered_matches)
# Print performance metrics
print_fuzzy_evaluation_metrics(annotation_list_cleaned, abstract_list_cleaned, filtered_matches, abstract_ids, abstract_unique_values,annotation_unique_values)

In [9]:
# export fuzzy match entities
export_entities_fuzzy(model_name, filtered_matches, abstract_list_cleaned, abstract_file_name)
# append performance metrics
export_performance_fuzzy(model_name, abstract_list_cleaned, annotation_unique_values, abstract_unique_values, num_common_elements_fuzzy, precision_fuzzy, recall_fuzzy, f1_score_fuzzy, char_diff, abstract_file_name)

Rows processed: 70489
Total rows processed: 70489
